In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline
try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

In [2]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [3]:
ele = pd.read_csv("data_ne.csv")
ad = pd.read_csv("ssc2020_annual_demand.csv")

In [4]:
ad_1 = ad.drop("Year",axis = 1)

In [5]:
annual_target = ad_1.sum(axis=1)

In [6]:
annual_total_0 = ele[["Usage_ma","Year"]].groupby("Year").sum()*0.0000036

In [7]:
annual_total = annual_total_0.reset_index()["Usage_ma"]

In [8]:
## True discrepancy
True_dis = annual_total - annual_target

In [9]:
True_dis

0     87.736925
1     90.673092
2     97.097832
3     71.695412
4     78.640293
5     46.833283
6     46.496174
7     28.401356
8     12.994808
9     19.435770
10     7.852422
11     8.293770
12    -3.557592
13     8.263089
dtype: float64

In [10]:
# unique year
uy = np.unique(ele["Year"])

In [11]:
ele_0 = ele

In [12]:
for i in range(len(uy)):
    ele_0.loc[ele["Year"] == uy[i],"Usage_ma"] -= True_dis[i]/(ele_0.loc[ele_0["Year"] == uy[i]].shape[0]*0.0000036)

In [13]:
annual_total_1 = ele_0[["Usage_ma","Year"]].groupby("Year").sum()*0.0000036

In [14]:
annual_total_1 = annual_total_1.reset_index()["Usage_ma"]

In [15]:
annual_total_1

0     460.1
1     461.7
2     468.0
3     472.1
4     469.3
5     488.4
6     454.5
7     483.5
8     496.3
9     489.2
10    498.8
11    495.0
12    496.8
13    484.9
Name: Usage_ma, dtype: float64

# LightGBM Implementation

In [16]:
ele_0 = ele_0.drop("Unnamed: 0",axis=1)
eval_arr = []

In [17]:
ele_0['lag1'] = ele_0['Usage_ma'].shift(1)

In [18]:
ele_0

,Date,Hour,Year,Month,time,Usage_ma,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_depth,cloud_cover,air_density,if_holiday,weekend,Max,Min,sta_dev,lag1
0,2003-01-01,1,2003,1,2003-01-01 01:00:00,11962.880370,0.0100,-1.720,0.0000,0.0000,0.0011,17.4309,0.3196,1.2612,True,False,18594.0,13236.0,1789.887629,NaN
1,2003-01-01,2,2003,1,2003-01-01 02:00:00,11497.880370,0.0022,-2.045,0.0000,0.0000,0.0010,17.4307,0.3167,1.2644,True,False,18594.0,13236.0,1789.887629,11962.880370
2,2003-01-01,3,2003,1,2003-01-01 03:00:00,11038.880370,0.0014,-2.402,0.0000,0.0000,0.0009,17.4304,0.2958,1.2669,True,False,18594.0,13236.0,1789.887629,11497.880370
3,2003-01-01,4,2003,1,2003-01-01 04:00:00,10456.880370,0.0013,-2.732,0.0000,0.0000,0.0008,17.4302,0.3745,1.2695,True,False,18594.0,13236.0,1789.887629,11038.880370
4,2003-01-01,5,2003,1,2003-01-01 05:00:00,10453.880370,0.0011,-3.083,0.0000,0.0000,0.0009,17.4300,0.6073,1.2724,True,False,18594.0,13236.0,1789.887629,10456.880370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122731,2016-12-31,20,2016,12,2016-12-31 20:00:00,15998.695064,0.1642,0.551,65.7947,394.8765,0.0845,16.9569,0.9761,1.2408,False,True,17340.0,12861.0,1318.496000,16659.695064
122732,2016-12-31,21,2016,12,2016-12-31 21:00:00,15396.695064,0.1857,0.455,29.8108,238.0150,0.0863,16.9597,0.9729,1.2407,False,True,17340.0,12861.0,1318.496000,15998.695064
122733,2016-12-31,22,2016,12,2016-12-31 22:00:00,14933.695064,0.2322,0.180,4.8503,52.9368,0.1055,16.9680,0.9628,1.2417,False,True,17340.0,12861.0,1318.496000,15396.695064
122734,2016-12-31,23,2016,12,2016-12-31 23:00:00,14496.695064,0.2462,-0.047,0.0047,0.0378,0.1173,16.9865,0.9353,1.2430,False,True,17340.0,12861.0,1318.496000,14933.695064


In [19]:
## lag extraction
ele_0['lag1'] = ele_0['Usage_ma'].shift(1)
ele_0['lag2'] = ele_0['Usage_ma'].shift(2)

ele_2 = ele_0.loc[:,["Year","Month","weekend","Hour","lag1","lag2","temperature","Usage_ma"]]

In [20]:
index_1 = [24*i for i in range(1,6,1)]

In [21]:
index_1

[24, 48, 72, 96, 120]

In [22]:
lag1_impute = ele_2.iloc[index_1]["lag1"].mean()

In [23]:
lag2_impute = ele_2.iloc[index_1]["lag2"].mean()

In [24]:
## time series imputation by auto regression
ele_2.loc[0,"lag1"] = lag1_impute
ele_2.loc[0,"lag2"] = lag2_impute
ele_2.loc[1,"lag2"] = lag1_impute

In [25]:
# cleaned data
ele_2.head()

,Year,Month,weekend,Hour,lag1,lag2,temperature,Usage_ma
0,2003,1,False,1,14083.88037,15180.48037,-1.720,11962.88037
1,2003,1,False,2,11962.88037,14083.88037,-2.045,11497.88037
2,2003,1,False,3,11497.88037,11962.88037,-2.402,11038.88037
3,2003,1,False,4,11038.88037,11497.88037,-2.732,10456.88037
4,2003,1,False,5,10456.88037,11038.88037,-3.083,10453.88037


## Random Search Optimization

In [26]:
from sklearn.model_selection import RandomizedSearchCV
## function of getting the optimized paramters and score
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
                                  n_jobs=-1, n_iter=nbr_iter, cv=10)
    
    #CV = Cross-Validation (here using Stratified KFold CV)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params, ht_score

### LightGBM hyperparameter optimization

In [ ]:
p_distr = {
    'learning_rate': [0.01,0.02,0.03,0.04,0.05],
    'num_leaves': [90,200,300,400,500],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'max_depth' : [5,6,7,8,9,10,11,12],
    'random_state' : [501], 
    'colsample_bytree' : [0.5,0.7,0.8],
    'subsample' : [0.5,0.7,0.8,0.9],
    'min_split_gain' : [0.01],
    'min_data_in_leaf':[10],
    }

X = ele_2.loc[:,["Year","Month","weekend","Hour","lag1","lag2","temperature"]]
y = ele_2.loc[:,"Usage_ma"]
est = lgb.LGBMRegressor()
nbr_iter = 20

random_params, random_score = hypertuning_rscv(est, p_distr, nbr_iter,X,y)

In [ ]:
random_params

In [ ]:
random_score

## lightGBM Coss validation

In [45]:
predict = []
for year in range(2003,2017) :
    train_X = ele_2.loc[ele_2["Year"]!= year,["Year","Month","weekend","Hour","lag1","lag2","temperature"]]
    train_y = ele_2.loc[ele_2["Year"]!= year,"Usage_ma"]
    
    test_X = ele_2.loc[ele_2["Year"] == year,["Year","Month","weekend","Hour","lag1","lag2","temperature"]]
    test_y = ele_2.loc[ele_2["Year"] == year,"Usage_ma"]
    
    
    
    if year!=2016:
        ## indexes that use the information of test data
        ind_1 = min(ele_2.loc[ele_2["Year"]==year+1,].index)
        ind_2 = min(ele_2.loc[ele_2["Year"]==year+1,].index)+1 
        
        
        train_X.iloc[ind_1, train_X.columns.get_loc('lag1')] = np.nan
        train_X.iloc[ind_1, train_X.columns.get_loc('lag2')] = np.nan
        
        train_X.iloc[ind_1, train_X.columns.get_loc('lag1')] = np.nan
        
        index_1 = [24*i for i in range(1,6,1)]
        
        lag1_impu = ele_2.iloc[index_1]["lag1"].mean()
        lag2_impu = ele_2.iloc[index_1]["lag2"].mean()
        
        ## time series imputation by auto regression
        train_X.loc[ind_1,"lag1"] = lag1_impu
        train_X.loc[ind_1,"lag2"] = lag2_impu
        train_X.loc[ind_2,"lag2"] = lag1_impu
    
    d_train = lgb.Dataset(train_X, label= train_y)
    
    params = {}
    params['learning_rate']= 0.04
    params['boosting_type']='gbdt'
    params['objective']='regression'
    params['metric']='l2'
    params['sub_feature']=0.7
    params['num_leaves']= 400
    params['min_data']=50
    params['max_depth']=12
    params['subsample']=0.8
    params['min_data_in_leaf']=10
    
    
    clf = lgb.train(params, d_train, 10000)
    #Prediction
    y_pred=clf.predict(test_X)
    predict.append(sum(y_pred)*0.0000036)

In [46]:
predict

[460.3040547216514,
 461.54243559815217,
 468.44854927373854,
 471.02951732779337,
 469.90352298318703,
 486.9347075715956,
 455.71738221810904,
 482.11647390529384,
 495.2668489064962,
 489.7789653155194,
 497.7521367627457,
 495.11789766240366,
 496.7661885454699,
 486.0850293613653]

In [47]:
abs(annual_total_1-predict).mean()

0.7533637798783478

In [42]:
random_params, random_score = hypertuning_rscv(est, p_distr, nbr_iter,X,y)

In [43]:
random_params

{'subsample': 0.8,
 'random_state': 501,
 'objective': 'regression',
 'num_leaves': 400,
 'min_split_gain': 0.01,
 'min_data_in_leaf': 10,
 'max_depth': 12,
 'learning_rate': 0.04,
 'colsample_bytree': 0.8,
 'boosting_type': 'gbdt'}

In [44]:
random_score

0.994110297065857

# XGBOOST Implementation

### XGBOOST hyperparameter optimization

In [44]:
# A parameter grid for XGBoost
p_distr ={
        'learning_rate': [0.01,0.02,0.03,0.04,0.05],
        'n_estimators': [100*i for i in range(1,11,1)],
        'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
        'gamma': [0, 1,3,5,7,10],
        'subsample': [0.6,0.7,0.8,0.9,1.0],
        'colsample_bytree': [0.6,0.7,0.8,0.9,1.0],
        'max_depth': [3,4,5,6,7,8,9,10],
        'eta': [0.1,0.2]
        }

X = ele_2.loc[:,["Year","Month","weekend","Hour","lag1","lag2","temperature"]]
y = ele_2.loc[:,"Usage_ma"]
est = xgb.XGBRegressor(objective ='reg:linear')
nbr_iter = 20

random_params, random_score = hypertuning_rscv(est, p_distr, nbr_iter,X,y)

[22:41:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [45]:
random_params

{'subsample': 0.8,
 'n_estimators': 1000,
 'min_child_weight': 2,
 'max_depth': 10,
 'learning_rate': 0.03,
 'gamma': 0,
 'eta': 0.1,
 'colsample_bytree': 0.8}

## XGBOOST Coss validation

In [47]:
predict = []
for year in range(2003,2017) :
    train_X = ele_2.loc[ele_2["Year"]!= year,["Year","Month","weekend","Hour","lag1","lag2","temperature"]]
    train_y = ele_2.loc[ele_2["Year"]!= year,"Usage_ma"]
    
    test_X = ele_2.loc[ele_2["Year"] == year,["Year","Month","weekend","Hour","lag1","lag2","temperature"]]
    test_y = ele_2.loc[ele_2["Year"] == year,"Usage_ma"]
    
    
    
    if year!=2016:
        ## indexes that use the information of test data
        ind_1 = min(ele_2.loc[ele_2["Year"]==year+1,].index)
        ind_2 = min(ele_2.loc[ele_2["Year"]==year+1,].index)+1 
        
        
        train_X.iloc[ind_1, train_X.columns.get_loc('lag1')] = np.nan
        train_X.iloc[ind_1, train_X.columns.get_loc('lag2')] = np.nan
        
        train_X.iloc[ind_1, train_X.columns.get_loc('lag1')] = np.nan
        
        index_1 = [24*i for i in range(1,6,1)]
        
        lag1_impu = ele_2.iloc[index_1]["lag1"].mean()
        lag2_impu = ele_2.iloc[index_1]["lag2"].mean()
        
        ## time series imputation by auto regression
        train_X.loc[ind_1,"lag1"] = lag1_impu
        train_X.loc[ind_1,"lag2"] = lag2_impu
        train_X.loc[ind_2,"lag2"] = lag1_impu
    
    params = {}
    params['learning_rate']= 0.05
    params['min_child_weight']=8
    params['colsample_bytree']=0.8
    params['max_depth']= 5
    params['subsample']=1.0
    params['gamma']=1.5
    params['objective']= 'reg:linear'
    
    
    xg_reg = xgb.XGBRegressor(n_estimators =1000,subsample=0.8,min_child_weight= 2,max_depth=10,learning_rate=0.03,gamma= 0,eta= 0.1,colsample_bytree= 0.8)
    
    xg_reg.fit(train_X,train_y)

    #Prediction
    y_pred=xg_reg.predict(test_X)
    predict.append(sum(y_pred)*0.0000036)

In [48]:
predict

[460.211384025,
 461.4807089460937,
 468.6698467722656,
 471.6310982800781,
 470.35029755742187,
 486.7293687046875,
 455.4524707453125,
 484.1612054648437,
 495.9418626597656,
 489.8927549285156,
 498.61638871523434,
 494.9891699625,
 495.7046078765625,
 485.8632921691406]

In [49]:
abs(annual_total_1-predict).mean()

0.6505747512654807